In [1]:
library(tidyverse)
library(tidymodels)
library(repr)
library(rvest)
library(stringr)
library(cowplot)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [2]:
tennis <- read_csv('https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS')
colnames(tennis) <- make.names(colnames(tennis))

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
tennis_data <- tennis |>
filter(Active == "Yes") |>
separate(col = Age, into = c("Age", "Birth.Date"), sep = " ") |>
separate(col = Current.Rank, into = c("Current.Rank", "Prev.Rank"), sep = " ") |>
separate(col = Current.Elo.Rank, into = c("Current.Elo.Rank", "Current.Elo.Rating"), sep = " ") |>
mutate(Current.Rank = as.numeric(Current.Rank),
       Current.Elo.Rating = as.numeric(gsub("[^0-9.-]", "", Current.Elo.Rating)),
       Age = as.numeric(Age),
       Prize.Money = as.numeric(gsub("[^0-9.-]", "", Prize.Money)),
       Money.Per.Season = Prize.Money/Seasons) |>
select(Name, Age, Current.Rank, Current.Elo.Rating, Money.Per.Season) |>
drop_na()
tennis_data

Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”


Name,Age,Current.Rank,Current.Elo.Rating,Money.Per.Season
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Lukas Lacko,32,178,1764,232969.1
Bernard Tomic,27,183,1826,553815.5
Hubert Hurkacz,22,31,1983,303431.4
⋮,⋮,⋮,⋮,⋮
Brayden Schnur,24,103,1627,103659.25
Mirza Basic,28,292,1764,149759.14
Dimitar Kuzmanov,26,324,1679,16608.88


In [4]:
set.seed(16)

tennis_split <- initial_split(tennis_data)
tennis_training <- training(tennis_split)
tennis_testing <- testing(tennis_split)